In [1]:
import torch
import time
from utils.utils import generate_mask, load_model, writeDACFile
from dataloader.dataset import onehot, getNumClasses
from utils.utils import interpolate_vectors

In [2]:
### params
cptnum=300
testsnd="bees"
checkpoint_dir='runs/newscratch'+ testsnd + "cond8"
fnamebase=testsnd+'.e248.l4.h8_chkpt_'+str(cptnum).zfill(4)
 
checkpoint_path = checkpoint_dir + '/' +  fnamebase  + '.pth' 

# for saving sound 
outdir=checkpoint_dir

DEVICE='cuda'

inference_steps=86*20  #86 frames per second

minpval=0
maxpval=1

In [3]:
torch.cuda.device_count()
torch.cuda.get_device_properties(0).total_memory/1e9

device = torch.device(DEVICE) # if the docker was started with --gpus all, then can choose here with cuda:0 (or cpu)
torch.cuda.device_count()
print(f'memeory on cuda 0 is  {torch.cuda.get_device_properties(0).total_memory/1e9}')

device

memeory on cuda 0 is  25.216745472


device(type='cuda')

In [4]:

model, Ti, vocab_size, num_codebooks, cond_size = load_model(checkpoint_path)
model.to(device);


Get a coder with embed_size=248. cond_size=8, max_len=430


In [5]:

if cond_size == 0 :
    inference_cond = None
else : 
    one_hot_fvector=onehot(testsnd)
    print(f'one_hot_fvector({testsnd}) = {one_hot_fvector}')

    cvect1=torch.cat((one_hot_fvector, torch.tensor([minpval])))
    cvect2=torch.cat((one_hot_fvector, torch.tensor([maxpval])))

    steps=[0,Ti+1*inference_steps//5, Ti+2*inference_steps//5, Ti+3*inference_steps//5, Ti+4*inference_steps//5, Ti+inference_steps]
    inference_cond=interpolate_vectors([cvect1,cvect1, cvect2, cvect2, cvect1, cvect1 ], steps) #length must cover staring context window+inf steps

    print(f'cvect1 = {cvect1}')
    print(f'cvect2 = {cvect2}')

    inference_cond=inference_cond.to(device)

    print(f'shape of inf_cond is  = {inference_cond.shape}') 

one_hot_fvector(bees) = tensor([0., 0., 0., 1., 0., 0., 0.])
cvect1 = tensor([0., 0., 0., 1., 0., 0., 0., 0.])
cvect2 = tensor([0., 0., 0., 1., 0., 0., 0., 1.])
shape of inf_cond is  = torch.Size([1, 1893, 8])


/dacsynthformer/utils/utils.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v_tensors = [torch.tensor(vec) for vec in v]


In [6]:
def inference(model, inference_cond, Ti, vocab_size, num_tokens, inference_steps, fname) :
    model.eval()
    mask = generate_mask(Ti, Ti).to(device)
    input_data = torch.randint(0, vocab_size, (1, Ti, num_tokens)).to(device)  # Smaller context window for inference
    predictions = []

    t0 = time.time()
    for i in range(inference_steps):  # Generate 100 tokens
        if cond_size == 0:
            output = model(input_data, None, mask) # step through 
        else : 
            output = model(input_data, inference_cond[:, i:Ti+i, :], mask) # step through

        # This takes the last vector of the sequence (the new predicted token stack) so has size(b,1,4,1024)
        # This it takes the max across the last dimension (scores for each element of the vocabulary (for each of the 4 tokens))
        # .max returns a duple of tensors, the first are the max vals (one for each token) and the second are the
        #        indices in the range of the vocabulary size. 
        # THAT IS, the 4 selected "best" tokens are taken independently
        next_token = output[:, -1, :, :].max(-1)[1]  # Greedy decoding for simplicity
        predictions.append(next_token)
        input_data = torch.cat([input_data, next_token.unsqueeze(1)], dim=1)[:, 1:]  # Slide window

    t1 = time.time()
    inf_time = t1-t0
    print(f'inference time for {inference_steps} steps, or {inference_steps/86} seconds of sound is {inf_time}' )

    dacseq = torch.cat(predictions, dim=0).unsqueeze(0).transpose(1, 2)
    if mask == None:
        writeDACFile(fname + '_unmasked', dacseq)
    else :
        writeDACFile(fname, dacseq)       

In [7]:
outfname=outdir+"/"+ "dacs" + "/" +  fnamebase+"_steps_"+str(inference_steps).zfill(4)+'.minpval_'+ f"{minpval:01.2f}" +'.maxpval_'+ f"{maxpval:01.2f}"
print(f'outfname is {outfname}')
inference(model, inference_cond, Ti, vocab_size, num_codebooks, inference_steps, outfname ) 


outfname is runs/newscratchbeescond8/dacs/bees.e248.l4.h8_chkpt_0300_steps_1720.minpval_0.00.maxpval_1.00
inference time for 1720 steps, or 20.0 seconds of sound is 5.111999034881592
Just so ya know, I had to create the path to save the file
